In [1]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [2]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import reload_lib as rl
import non_bio_co2 as nbc


lib = [s, lc, lp, nbc]

In [3]:
from copy import deepcopy as dc

In [4]:
import bw2io as bi
import bw2data as bd
import pandas as pd

In [5]:
bd.projects.set_current("single use vs multi use")


In [6]:
if any("biosphere" in db for db in bd.databases):
    print('Biosphere is already present in the project.')
else:
    bi.bw2setup()

Biosphere is already present in the project.


In [7]:
if 'ev391cutoff' in bd.databases and 'ev391consq' in bd.databases and 'ev391apos' in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    # APOS
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets", db_name='ev391apos') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # Consequential
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets", db_name='ev391consq') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # cut-off
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets", db_name='ev391cutoff') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

Ecoinvent 3.9.1 is already present in the project.


In [8]:
bd.databases

Databases dictionary with 10 object(s):
	biosphere3
	case1_apos
	case1_consq
	case1_cut_off
	case2_apos
	case2_consq
	case2_cut_off
	ev391apos
	ev391consq
	ev391cutoff

In [9]:
system_path = [r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case1.xlsx', 
              r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\case2.xlsx']

for path in system_path:    
    db_path = path 
    sheet_name = ['ev391apos', 'ev391consq', 'ev391cutoff'] # Replace with your actual sheet name

    for db in sheet_name:
        # print(idx+1, db)
        # Read the Excel file
        data = pd.read_excel(db_path, sheet_name=db)
        # Save the data to a temporary file that can be used by ExcelImporter
        temp_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\temp.xlsx'
        data.to_excel(temp_path, index=False)

        # if data.columns[1] not in bd.databases:

        # Use the temporary file with ExcelImporter
        imp = bi.ExcelImporter(temp_path)  # the path to your inventory excel file
        imp.apply_strategies()

        # # List of databases to match

        # Loop through each database and match
        print(f"Matching database: {db}")
        imp.match_database(db, fields=('name', 'unit', 'location', 'reference product'))
        print(f"Unlinked items after matching {db}: {list(imp.unlinked)}")

        # Match without specifying a database
        imp.match_database(fields=('name', 'unit', 'location'))

        # Generate statistics and write results
        imp.statistics()
        imp.write_excel(only_unlinked=True)
        unlinked_items = list(imp.unlinked)
        imp.write_database()

        # Print unlinked items if needed
        print(unlinked_items)
        print(f'{data.columns[1]} is loaded into the database')
        # else:
        #     print(f'{data.columns[1]} already exist')

Extracted 1 worksheets in 0.04 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 3.27 seconds
Matching database: ev391apos
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Unlinked items after matching ev391apos: []
Applying strategy: link_iterable_by_fields
17 datasets
150 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\ruw\AppData\Local\pylca\Brightway3\single-use-vs-multi-use.0e7e263503916b8fcd098a9e51faa14f\output\db-matching-case1_apos-unlinked.xlsx


0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 10:01:13
  Finished: 01/09/2025 10:01:13
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.92
Created database: case1_apos
[]
case1_apos is loaded into the database
Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: conv

...

Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 10:01:38
  Finished: 01/09/2025 10:01:38
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.08
Created database: case2_consq
[]
case2_consq is loaded into the database
Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: co

Writing activities to SQLite3 database:


Unlinked items after matching ev391cutoff: [{'name': 'autoclave', 'amount': 0.0277777777777777, 'reference product': 'autoclave operation', 'location': 'GLO', 'unit': 'unit', 'type': 'technosphere', 'database': 'case2_apos'}, {'name': 'scalpel', 'amount': 1, 'reference product': 'scalpel', 'location': 'GLO', 'unit': 'unit', 'categories': 0, 'type': 'production', 'database': 'case2_apos'}]
Applying strategy: link_iterable_by_fields
22 datasets
120 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\ruw\AppData\Local\pylca\Brightway3\single-use-vs-multi-use.0e7e263503916b8fcd098a9e51faa14f\output\db-matching-case2_cut_off-unlinked.xlsx


0% [######################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/09/2025 10:01:43
  Finished: 01/09/2025 10:01:43
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.06
Created database: case2_cut_off
[]
case2_cut_off is loaded into the database


In [30]:
database = "case1_cut_off"
db = bd.Database(database)
flow = []
if 'case1' in str(db):
    for act in db:
        temp = act['name']
        # print(flow)
        if ('H2' in temp  or 'H4' in temp) and ('SU' in temp or 'REC' in temp) and temp not in flow:
           flow.append(temp) 
        elif 'alubox' in temp and '+' in temp and 'eol' not in temp.lower():
            flow.append(temp) 
    flow.sort()
elif 'case2' in str(db):
    for act in db:
        temp = act['name']
        if temp == 'SUD' or temp == 'MUD':
            flow.append(temp)
    flow.sort()
    flow.reverse()

flow

['H200 REC',
 'H200 SU',
 'H400 REC',
 'H400 SU',
 'alubox (large + dishwasher)',
 'alubox (large + wipe)',
 'alubox (small + dishwasher)',
 'alubox (small + wipe)']

In [31]:
flows = flow

In [32]:
procces_keys = {key: None for key in flows}

size = len(flows)

for act in db:
        for proc in range(size):
            if act['name'] == flows[proc]:
                procces_keys[act['name']] = act['code']

procces_keys

{'H200 REC': 'b1178f93d030e078ebbae2af3fe76929',
 'H200 SU': '1aeeffce0d2b541414350222ee2e7246',
 'H400 REC': '6c866d09cde0b3fa0daf28c657e28dc7',
 'H400 SU': '7b9d008c491d64e1b98d8809c2ec8331',
 'alubox (large + dishwasher)': '0a97ff39ffed072e23f98d2b27ca8c02',
 'alubox (large + wipe)': '65231d0406d7ca520fa7f8ab0ef57758',
 'alubox (small + dishwasher)': '7807e75f1bc0c9c573389f20179d9db1',
 'alubox (small + wipe)': 'b3dcf85f3c8b1ed940d2c6043a2f33b9'}

In [33]:
process = []
for key, item in procces_keys.items():
        try:
            process.append(db.get(item))
        except KeyError:
            print(f"Process with key '{item}' not found in the database '{db}'")
            process = None
process

['H200 REC' (unit, GLO, None),
 'H200 SU' (unit, GLO, None),
 'H400 REC' (unit, GLO, None),
 'H400 SU' (unit, GLO, None),
 'alubox (large + dishwasher)' (unit, GLO, None),
 'alubox (large + wipe)' (unit, GLO, None),
 'alubox (small + dishwasher)' (unit, GLO, None),
 'alubox (small + wipe)' (unit, GLO, None)]

In [34]:
product_details = {}
product_details_code = {}

# Obtaining the subsubprocess'
if process:
    for proc in process:
        product_details[proc['name']] = []
        product_details_code[proc['name']] = []

        for exc in proc.exchanges():
            if exc['type'] == 'technosphere' or exc['type'] == 'biosphere':
            # if 'Use' in exc.output['name'] and exc['type'] == 'biosphere':
                product_details[proc['name']].append({exc.input['name']: [exc['amount'], exc.input]})
                # if exc.input in eidb or exc.input in eidb_db or exc.input in eidb_cyl:
                #     product_details_code[proc['name']].append([exc.output, exc.output['name'], exc.output['code'], exc['amount']])
            # elif exc['type'] == 'technosphere':
            #     product_details[proc['name']].append({exc.input['name']: [exc['amount'], exc.input]})


In [35]:
FU = {key: {} for key in product_details.keys()}
# Creating the FU to calculate for
for key, item in product_details.items():
    for idx in item:
        for n, m in idx.items():
            FU[key].update({m[1]: m[0]})


In [36]:
FU

{'H200 REC': {'H200' (unit, GLO, None): 1,
  'autoclave' (unit, GLO, None): 0.08333333333333333,
  'treatment of waste polypropylene, municipal incineration with fly ash extraction' (kilogram, CH, None): -0.0107,
  'treatment of waste polyethylene, municipal incineration with fly ash extraction' (kilogram, CH, None): -0.0109,
  'market for electricity, high voltage' (kilowatt hour, DK, None): -0.022063716860625,
  'mixed heating grid' (megajoule, GLO, None): -0.39714690349125,
  'market for polypropylene, granulate' (kilogram, GLO, None): -0.0604,
  'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None): -0.00618,
  'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None): 0.00727,
  'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None): 0.071,
  'treatment of waste paper, unsorted, sortin

In [37]:
unique_process_index = []
uniquie_process = []

for act, exc in FU.items():
    for proc in exc.keys():
        if str(proc) not in unique_process_index:
            unique_process_index.append(str(proc))
            uniquie_process.append(proc)
unique_process_index.sort()


In [38]:
FU1 = []
for upi in unique_process_index:
    for proc in uniquie_process:
        if upi == f'{proc}':
            FU1.append({proc: 1})
            print(proc[1])


e28608ef0c45aca4f0de958b230e98ca
ad663ab3609e6ff84869860b0e67df74
0f35358b8e575a4211da7223c7c385ee
81674eabe949993b3213ad70bc74b631
2ebeb132d01ea64fe7eb373814605e57
fe46c28da468b81defdd4f4d50915a9e
a133f0dc517dbc09325c8432f664f4d9
61863071bd168004eb9af9df563661b5
d508253b15716f01116c946e9f0ac027
bac928a8c48ad4711bd0e97cdcc3ab15
c2f45445bccf5e1a4ecc32e6312b6128
bdb6614aecb3510c2ea062aac9744c6c
ae201ebdde0dba35fd579e7e9e00b87b
290d83cc6e3555585f970229186e386a
e7e9803e3220fcca91d940d242016e95
6a4fdd1e422f2d95fd4e5b9e43324ac8
3cf0e8a56c707cdce7885e5a7435312b
05f3bcea2adc0750ebeb720a82945eac
b8eec0ebb20533ef6d2728fea9438fb1
573637ccc8152f031c01814e0a9fecdf


In [39]:
FU1

[{'H200' (unit, GLO, None): 1},
 {'H400' (unit, GLO, None): 1},
 {'alubox (large)' (unit, GLO, None): 1},
 {'alubox (small)' (unit, GLO, None): 1},
 {'autoclave' (unit, GLO, None): 1},
 {'market for aluminium, cast alloy' (kilogram, GLO, None): 1},
 {'market for electricity, high voltage' (kilowatt hour, DK, None): 1},
 {'market for polypropylene, granulate' (kilogram, GLO, None): 1},
 {'mechanical disinfection' (unit, GLO, None): 1},
 {'mixed heating grid' (megajoule, GLO, None): 1},
 {'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None): 1},
 {'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None): 1},
 {'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None): 1},
 {'treatment of aluminium scrap, post-consumer, prepared for recycling, at remelter' (kilogram, RER, None): 1},
 {'treatmen

In [40]:
import bw2calc as bc

In [41]:
impact_categories = lc.lcia_method('recipe')
if type(impact_categories) == tuple:
    impact_categories = [ic for ic in impact_categories]
df_unique = pd.DataFrame(0, index=unique_process_index, columns=impact_categories, dtype=object)
# unique_process_results = {}
# calc_count = 1
# total_calculations = len(uniquie_process_dct)*len(impact_categories)
print(f'Calculating for {len(impact_categories)} methods and {len(FU1)} activities : Total calculations {len(impact_categories) * len(FU1)}' )
bd.calculation_setups['calc_setup'] = {'inv':FU1, 'ia': impact_categories}
mylca = bc.MultiLCA('calc_setup')
res = mylca.results
for col, arr in enumerate(res):
    for row, val in enumerate(arr):
        df_unique.iat[col, row] = val

df_unique

Recipe is selected
Calculating for 21 methods and 20 activities : Total calculations 420


,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: natural resources, natural resources)"
"'H200' (unit, GLO, None)",0.000717,0.247601,0.007639,0.010073,0.67358,0.149743,0.000055,0.000008,0.009803,0.15266,...,0.004108,0.002435,0.0,0.000309,0.000581,0.000616,0.002001,0.0,0.0,0.057155
"'H400' (unit, GLO, None)",0.002052,0.708573,0.021853,0.028816,1.926022,0.428409,0.000157,0.000022,0.028067,0.43711,...,0.011974,0.006964,0.0,0.000883,0.001664,0.001763,0.005759,0.0,0.000001,0.163507
"'alubox (large)' (unit, GLO, None)",0.000563,0.129929,0.010888,0.01382,0.318967,0.029396,0.000048,0.000005,0.022852,0.142855,...,0.002342,0.002122,0.0,0.000244,0.000326,0.000333,0.001337,0.0,0.0,0.006459
"'alubox (small)' (unit, GLO, None)",0.000336,0.077379,0.006497,0.008246,0.189747,0.017439,0.000028,0.000003,0.013643,0.085121,...,0.001396,0.001261,0.0,0.000146,0.000194,0.000198,0.000794,0.0,0.0,0.003814
"'autoclave' (unit, GLO, None)",0.009221,2.613781,0.523706,0.659468,27.284128,0.672673,0.002558,0.001769,0.244313,7.418088,...,0.485317,0.056203,0.000003,0.00319,0.005522,0.00566,0.047911,0.0,0.000007,0.141107
"'market for aluminium, cast alloy' (kilogram, GLO, None)",0.025302,5.8042,3.252091,3.926636,13.224076,1.237342,0.001835,0.000142,1.104179,9.089712,...,0.077364,0.108836,0.000001,0.011103,0.015381,0.015635,0.035214,0.0,0.000018,0.26386
"'market for electricity, high voltage' (kilowatt hour, DK, None)",0.000504,0.20841,0.0125,0.016302,0.624514,0.054716,0.000163,0.000012,0.016628,0.294784,...,0.043494,0.002096,0.0,0.00017,0.000409,0.000418,0.003495,0.0,0.0,0.01032
"'market for polypropylene, granulate' (kilogram, GLO, None)",0.006304,2.378756,0.054088,0.072081,5.129597,1.713174,0.000389,0.000039,0.085705,1.151823,...,0.018377,0.020503,0.0,0.002389,0.00493,0.005279,0.017527,0.0,0.00

In [42]:
for col, impact in enumerate(impact_categories):
            for proc, fu in FU.items():
                for key, item in fu.items():
                    print(key,item)
                


'H200' (unit, GLO, None) 1
'autoclave' (unit, GLO, None) 0.08333333333333333
'treatment of waste polypropylene, municipal incineration with fly ash extraction' (kilogram, CH, None) -0.0107
'treatment of waste polyethylene, municipal incineration with fly ash extraction' (kilogram, CH, None) -0.0109
'market for electricity, high voltage' (kilowatt hour, DK, None) -0.022063716860625
'mixed heating grid' (megajoule, GLO, None) -0.39714690349125
'market for polypropylene, granulate' (kilogram, GLO, None) -0.0604
'polyethylene, high density, granulate, recycled to generic market for high density PE granulate' (kilogram, Europe without Switzerland, None) -0.00618
'polyethylene production, high density, granulate, recycled' (kilogram, Europe without Switzerland, None) 0.00727
'polyethylene terephthalate production, granulate, amorphous, recycled' (kilogram, Europe without Switzerland, None) 0.071
'treatment of waste paper, unsorted, sorting' (kilogram, RoW, None) -0.00316
'H200' (unit, GLO, N

In [43]:
functional_unit = FU
df_res= pd.DataFrame(0, index=flows, columns=impact_categories, dtype=object)

for col in impact_categories:
    for i, row in df_res.iterrows():
        row[col] = []

for col, impact in enumerate(impact_categories):
            for proc, fu in functional_unit.items():
                # print(proc)
                for key, item in fu.items():
                    exc = str(key)
                    val = float(item)
                    factor = df_unique.at[exc, impact]
                    impact_value = val * factor
                    df_res.at[proc, impact].append([exc, impact_value])
df_res        


,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, acidification: terrestrial, terrestrial acidification potential (TAP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, climate change, global warming potential (GWP1000))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: freshwater, freshwater ecotoxicity potential (FETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: marine, marine ecotoxicity potential (METP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ecotoxicity: terrestrial, terrestrial ecotoxicity potential (TETP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, energy resources: non-renewable, fossil, fossil fuel potential (FFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: freshwater, freshwater eutrophication potential (FEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, eutrophication: marine, marine eutrophication potential (MEP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: carcinogenic, human toxicity potential (HTPc))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, human toxicity: non-carcinogenic, human toxicity potential (HTPnc))",...,"(ReCiPe 2016 v1.03, midpoint (H) Runes edition, land use, agricultural land occupation (LOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, material resources: metals/minerals, surplus ore potential (SOP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, ozone depletion, ozone depletion potential (ODPinfinite))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, particulate matter formation, particulate matter formation potential (PMFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: human health, photochemical oxidant formation potential: humans (HOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, photochemical oxidant formation: terrestrial ecosystems, photochemical oxidant formation potential: ecosystems (EOFP))","(ReCiPe 2016 v1.03, midpoint (H) Runes edition, water use, water consumption potential (WCP))","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (H) Runes edition, total: natural resources, natural resources)"
H200 REC,"[['H200' (unit, GLO, None), 0.0007172858214633...","[['H200' (unit, GLO, None), 0.2476010125521856...","[['H200' (unit, GLO, None), 0.0076389196226727...","[['H200' (unit, GLO, None), 0.0100727393929656...","[['H200' (unit, GLO, None), 0.6735796752987794...","[['H200' (unit, GLO, None), 0.149743328893983]...","[['H200' (unit, GLO, None), 5.476664503214043e...","[['H200' (unit, GLO, None), 7.73726858284468e-...","[['H200' (unit, GLO, None), 0.0098025200510770...","[['H200' (unit, GLO, None), 0.1526597166183040...",...,"[['H200' (unit, GLO, None), 0.0041080420182904...","[['H200' (unit, GLO, None), 0.0024345451694959...","[['H200' (unit, GLO, None), 4.656145841239028e...","[['H200' (unit, GLO, None), 0.0003086847791470...","[['H200' (unit, GLO, None), 0.0005812956178185...","[['H200' (unit, GLO, None), 0.0006158725330556...","[['H200' (unit, GLO, None), 0.0020008693134336...","[['H200' (unit, GLO, None), 1.0122212290609556...","[['H200' (unit, GLO, None), 4.961592795305003e...","[['H200' (unit, GLO, None), 0.0571551879927072..."
H200 SU,"[['H200' (unit, GLO, None), 0.0007172858214633...","[['H200' (unit, GLO, None), 0.2476010125521856...","[['H200' (unit, GLO, None), 0.0076389196226727...","[['H200' (unit, GLO, None), 0.0100727393929656...","[['H200' (unit, GLO, None), 0.6735796752987794...","[['H200' (unit, GLO, None), 0.149743328893983]...","[['H200' (unit, GLO, None), 5.476664503214043e...","[['H200' (unit, GLO, None), 7.73726858284468e-...","[['H200' (unit, GLO, None), 0.0098025200510770...","[['H200' (unit, GLO, None), 0.1526597166183040...",...,"[['H200' (unit, GLO, None), 0.0041080420182904...","[['H200' (unit, GLO, None), 0.0024345451694959...","[['H200' (uni

In [55]:
idx_dct = {}
idx_lst = df_res.index
if 'case1' in database:
    plc_lst = [1, 0, 5, 4, 6, 7, 2, 3]

# Letting the user decide the new order of the index
for plc, idx in enumerate(df_res.index):
    idx_dct[idx] = plc_lst[plc]



In [59]:

# Creating the new index list
idx_lst = [''] * len(idx_dct)
for key, item in idx_dct.items():
    idx_lst[item] = key

impact_category = df_res.columns
df_rearranged = pd.DataFrame(0, index=idx_lst, columns=impact_category, dtype=object)


# Arranging the dataframe to the new dataframe
for icol, col in enumerate(impact_category):
    for row_counter, idx in enumerate(df_rearranged.index):
        rearranged_val = df_res.at[idx, col] # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html#pandas.DataFrame.at
        df_rearranged.iloc[row_counter, icol] = rearranged_val
